In [134]:
import pandas as pd
import numpy as np
import ast
from collections import Counter, OrderedDict
import matplotlib.pyplot as plt
df = pd.read_csv("songs_and_chords.csv")
data = [ast.literal_eval(chords_string) for chords_string in df["encoded_chords"]]

Remove too short songs

In [135]:
print(len(data))
SAMPLE_LEN = 8
data = [chords for chords in data if len(chords)>SAMPLE_LEN]
print(len(data))

2137
2132


Sampling dataset 8 chords before and 9'th cord as a value to predict

In [197]:
x_samples = []
y_samples = []
for chords in [data[5]]:
    for i in range(len(chords)):
        if i < len(chords) - SAMPLE_LEN:
            x_samples.append(chords[i : i + SAMPLE_LEN - 1])
        else:
            x_samples.append(chords[i : len(chords) - 1] + chords[: (i + SAMPLE_LEN) % len(chords)])
        y_samples.append(chords[(i + SAMPLE_LEN) % len(chords)])    
    break
x_samples = np.array(x_samples)
y_samples = np.array(y_samples)
print(x_samples.shape, y_samples.shape)
import music21
for chord in x_samples[0]:
    print(len(chord))
    print(type(list(chord)))
    print(chord)
    # decode
#     music21.chord.Chord([1,2,3])
    break
# names = [music21.chord.Chord(list(chord)).commonName for chord in x_samples[0]]
# print(names)
# print(music21.chord.Chord(y_samples[0]).commonName)

(36, 7, 12) (36, 12)
12
<class 'list'>
[1 0 1 1 0 1 0 1 0 0 1 0]


Long Short-Term Memory, or LSTM, recurrent neural networks expect three-dimensional input in the Keras Python deep learning library.
LSTMs expect 3D input, and it can be challenging to get your head around this the first time.
LSTMs don’t like sequences of more than 200-400 time steps, so the data will need to be split into samples.

If you have a long sequence of thousands of observations in your time series data, you must split your time series into samples and then reshape it for your LSTM model.

In [3]:

# LSTM for sequence classification in the IMDB dataset
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))



17465344/17464789 [==============================] - 3s 0us/step
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               53200     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
___________________________________________________